# Student RAG Project - Guided Implementation

## Welcome!

In this project, you'll build a **RAG (Retrieval-Augmented Generation)** system that can answer questions about your documents.

### What You'll Learn:
- ✅ File I/O (reading documents)
- ✅ String manipulation (text chunking)
- ✅ Functions and parameters
- ✅ Lists and dictionaries
- ✅ Loops and conditionals
- ✅ Basic calculations and statistics

### What's Provided for You:
- ✅ Embedding model (converts text to numbers)
- ✅ Vector database (stores and searches embeddings)
- ✅ LLM connection (generates answers)

### Your Tasks:
You'll complete **TODO sections** marked with `# TODO:` comments.

Let's get started! 🚀

---
## Setup: Import Libraries

In [2]:
# Import the pre-built helper module
from rag_helpers import (
    EmbeddingModel,
    VectorDatabase,
    LLM,
    Timer,
    print_separator,
    print_search_results,
    print_rag_answer,
    check_setup
)

# Import standard Python libraries you'll use
from pathlib import Path
from typing import List, Dict
import json

# Check if everything is installed correctly
check_setup()

c:\Users\Mejia\.conda\envs\rag3_313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking setup...
✓ chromadb is installed
✓ sentence_transformers is installed
✓ requests is installed

✓ All required packages are installed!
You're ready to start!


True

---
## Configuration

Set up the basic settings for your RAG system.

In [3]:
# TODO: Change this to point to YOUR documents folder
DOCS_FOLDER = "./docs/text"

# Chunking settings (you can experiment with these!)
CHUNK_SIZE = 500      # How many characters per chunk
OVERLAP = 50          # How many characters overlap between chunks

# How many results to retrieve for each query
TOP_K = 3

print(f"Configuration:")
print(f"  Documents folder: {DOCS_FOLDER}")
print(f"  Chunk size: {CHUNK_SIZE} characters")
print(f"  Overlap: {OVERLAP} characters")
print(f"  Top-K results: {TOP_K}")

Configuration:
  Documents folder: ./docs/text
  Chunk size: 500 characters
  Overlap: 50 characters
  Top-K results: 3


---
## TODO #1: Document Loading

**Your Task:** Write a function to load all text files from a folder.

**What to do:**
1. Loop through all `.txt` files in the folder
2. Read each file's content
3. Store the content and filename in a dictionary
4. Return a list of these dictionaries

**Python concepts:** File I/O, loops, dictionaries, lists

In [4]:
# 🛠 Debug + Load Documents Cell

import os
from pathlib import Path

print("Working directory:", os.getcwd())

# Check some common candidate paths
candidate_paths = ["docs/text", "./docs/text", "docs", "./docs"]
print("\nChecking candidate paths for .txt files:")
for cp in candidate_paths:
    folder = Path(cp)
    print(f"\nPath: {cp}")
    print("  Exists?:", folder.exists())
    if folder.exists():
        txt_files = list(folder.glob("*.txt"))
        print("  .txt files found:", txt_files)
    else:
        print("  .txt files found: []")

# Try to auto-select a good DOCS_FOLDER based on what we just saw
if Path("docs/text").exists() and list(Path("docs/text").glob("*.txt")):
    DOCS_FOLDER = "docs/text"
elif Path("./docs/text").exists() and list(Path("./docs/text").glob("*.txt")):
    DOCS_FOLDER = "./docs/text"
elif Path("docs").exists() and list(Path("docs").glob("*.txt")):
    DOCS_FOLDER = "docs"
elif Path("./docs").exists() and list(Path("./docs").glob("*.txt")):
    DOCS_FOLDER = "./docs"

print(f"\nUsing DOCS_FOLDER = {DOCS_FOLDER!r}")

# Now call your existing load_documents() with the detected folder
documents = load_documents(DOCS_FOLDER)

# Display first document (if any were loaded)
if documents:
    print(f"\nFirst document: {documents[0]['filename']}")
    print(f"Content preview: {documents[0]['content'][:200]}...")
else:
    print("⚠️  No documents loaded! Check your folder path again.")


Working directory: c:\Users\Mejia\Dev\Rag\IS640_rag_project

Checking candidate paths for .txt files:

Path: docs/text
  Exists?: True
  .txt files found: [WindowsPath('docs/text/arrozConGandules.txt'), WindowsPath('docs/text/coquito.txt'), WindowsPath('docs/text/empanadilla.txt'), WindowsPath('docs/text/panDeAgua.txt'), WindowsPath('docs/text/pastelon.txt'), WindowsPath('docs/text/pernil.txt'), WindowsPath('docs/text/Recipes.txt'), WindowsPath('docs/text/sample1.txt'), WindowsPath('docs/text/sofrito.txt')]

Path: ./docs/text
  Exists?: True
  .txt files found: [WindowsPath('docs/text/arrozConGandules.txt'), WindowsPath('docs/text/coquito.txt'), WindowsPath('docs/text/empanadilla.txt'), WindowsPath('docs/text/panDeAgua.txt'), WindowsPath('docs/text/pastelon.txt'), WindowsPath('docs/text/pernil.txt'), WindowsPath('docs/text/Recipes.txt'), WindowsPath('docs/text/sample1.txt'), WindowsPath('docs/text/sofrito.txt')]

Path: docs
  Exists?: True
  .txt files found: []

Path: ./docs
  Exists?

---
## TODO #2: Text Chunking Function

**Your Task:** Write a function to split long text into smaller chunks with overlap.

**Why?** Long documents are too big for embeddings. We need to split them into smaller pieces.

**What to do:**
1. Start at the beginning of the text
2. Take a chunk of `chunk_size` characters
3. Move forward by `chunk_size - overlap` characters
4. Repeat until you reach the end

**Python concepts:** String slicing, loops, lists

In [ ]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """
    Split text into overlapping chunks.
    """
    chunks = []
    position = 0

    # Safety: ensure overlap is smaller than chunk size
    step = chunk_size - overlap
    if step <= 0:
        raise ValueError("Overlap must be smaller than chunk_size")

    # Main loop
    while position < len(text):
        chunk = text[position : position + chunk_size]

        # Only add non-empty chunks
        if chunk.strip():
            chunks.append(chunk)

        # Move forward
        position += step

    return chunks





---
## TODO #3: Process All Documents into Chunks

**Your Task:** Use your chunking function to split ALL documents into chunks and create metadata.

**What to do:**
1. Loop through each document
2. Chunk the document's content
3. For each chunk, create metadata (which file it came from, which chunk number)
4. Store everything in a list

**Python concepts:** Nested loops, dictionaries, enumerate

In [23]:
    for doc in documents:
        # Get document content and filename
        content = doc['content']
        filename = doc['filename']

        # Chunk the document
        chunks = chunk_text(content, chunk_size=chunk_size, overlap=overlap)

        # For each chunk, create metadata and store results
        for idx, chunk in enumerate(chunks):
            # Add chunk text to list
            chunk_texts.append(chunk)

            # Create metadata dictionary
            metadata = {
                'source': filename,   # which file it came from
                'chunk_id': idx       # which chunk number within that file
            }

            # Add metadata to list
            chunk_metadatas.append(metadata)


NameError: name 'chunk_size' is not defined

---
## Pre-Built: Create Embeddings and Store in Database

This part uses the pre-built helpers. Just run these cells - no coding needed! ✨

In [22]:


# Initialize the embedding model (pre-built)
print("Initializing embedding model...")
embedder = EmbeddingModel()

# Create embeddings for all chunks (pre-built)
print("\nCreating embeddings...")
embeddings = embedder.embed_multiple(chunk_texts)
print(f"✓ Created {len(embeddings)} embeddings")



Initializing embedding model...
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
✓ Model loaded!

Creating embeddings...


NameError: name 'chunk_texts' is not defined

In [ ]:
# Initialize vector database (pre-built)
print("Initializing vector database...")
vector_db = VectorDatabase()

# Add chunks to database (pre-built)
print("\nAdding chunks to database...")
vector_db.add_chunks(chunk_texts, embeddings, chunk_metadatas)

In [ ]:
# Initialize LLM connection (pre-built)
print("Connecting to Ollama LLM...")
llm = LLM()

# Test the connection
print("\nTesting LLM connection...")
if llm.test_connection():
    print("✓ LLM is working!")
else:
    print("⚠️  LLM connection failed! Make sure Docker container is running.")

---
## TODO #4: RAG Query Function

**Your Task:** Write the main RAG function that ties everything together!

**What to do:**
1. Embed the user's question
2. Search the database for similar chunks
3. Build a prompt with the retrieved context
4. Ask the LLM to answer based on the context
5. Return the answer and metadata

**Python concepts:** Functions, string formatting, dictionaries

In [ ]:
def rag_query(question: str, top_k: int = 3) -> Dict:
    """
    Answer a question using RAG (Retrieval-Augmented Generation).

    Args:
        question: The user's question
        top_k: How many chunks to retrieve

    Returns:
        Dictionary with:
        - 'question': the original question
        - 'answer': the LLM's answer
        - 'sources': list of source filenames
        - 'contexts': list of retrieved chunks
        - 'time': how long it took
    """

    # Start timer
    timer = Timer()
    timer.start()

    # TODO: Implement the RAG pipeline!
    # HINTS:
    # 1. Embed the question using: embedder.embed_text(question)
    # 2. Search database using: vector_db.search(query_embedding, top_k)
    # 3. Extract retrieved chunks and metadata from search results:
    #    - retrieved_chunks = results['documents'][0]
    #    - retrieved_metadata = results['metadatas'][0]
    # 4. Build context by joining chunks with newlines
    # 5. Create prompt (template below)
    # 6. Generate answer using: llm.generate_answer(prompt)
    # 7. Extract source filenames from metadata
    # 8. Return everything in a dictionary

    # Step 1: Embed question
    query_embedding = None  # Your code here

    # Step 2: Search database
    results = None  # Your code here

    # Step 3: Extract results
    retrieved_chunks = None  # Your code here
    retrieved_metadata = None  # Your code here

    # Step 4: Build context
    context = None  # Your code here (join chunks with '\n\n')

    # Step 5: Create prompt (use this template)
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""

    # Step 6: Generate answer
    answer = None  # Your code here

    # Step 7: Extract sources
    sources = []  # Your code here (get 'source' from each metadata dict)

    # Stop timer
    elapsed_time = timer.stop()

    # Step 8: Return results
    return {
        'question': question,
        'answer': answer,
        'sources': sources,
        'contexts': retrieved_chunks,
        'time': elapsed_time
    }


print("✓ RAG query function defined!")

---
## Test Your RAG System!

Let's try asking some questions!

In [ ]:
# Test question 1
result = rag_query("What are the attendance rules?")

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Try your own question!
my_question = "What happens if you cheat?"  # Change this!

result = rag_query(my_question)
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

---
## TODO #5: Create Test Dataset

**Your Task:** Create a list of test questions to evaluate your RAG system.

**What to do:**
1. Think of 10 questions your documents can answer
2. For each question, write the expected answer
3. Store them in a structured format

**Python concepts:** Lists, dictionaries, data structures

In [ ]:
# TODO: Create your test dataset!
# HINTS:
# Create a list of dictionaries
# Each dictionary should have:
#   - 'question': the test question
#   - 'expected_answer': what you expect the answer to include
#   - 'category': type of question (factual, inferential, etc.)

test_questions = [
    # Example (replace with your own!):
    {
        'question': 'What are the attendance rules?',
        'expected_answer': 'Students must attend all classes, excused absences need parent notification',
        'category': 'factual'
    },
    # Add 9 more questions here!
    # Your code here:

]

print(f"✓ Created {len(test_questions)} test questions")
print(f"\nExample question:")
print(f"  Q: {test_questions[0]['question']}")
print(f"  Expected: {test_questions[0]['expected_answer']}")

---
## TODO #6: Calculate Evaluation Metrics

**Your Task:** Write functions to measure how well your RAG system performs.

**Python concepts:** Functions, calculations, statistics

In [ ]:
def calculate_average_latency(results: List[Dict]) -> float:
    """
    Calculate average response time.

    Args:
        results: List of result dictionaries (each has 'time' field)

    Returns:
        Average time in seconds
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Extract all 'time' values from results
    # 2. Sum them up
    # 3. Divide by the number of results
    # 4. Return the average

    # Your code here:
    total_time = 0
    # Calculate sum and average

    return 0.0  # Replace with your calculation


def count_successful_retrievals(results: List[Dict]) -> int:
    """
    Count how many queries successfully retrieved context.

    Args:
        results: List of result dictionaries

    Returns:
        Number of successful retrievals
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Start with count = 0
    # 2. For each result:
    #    - Check if 'contexts' is not empty
    #    - If yes, increment count
    # 3. Return count

    # Your code here:
    count = 0
    # Count successful retrievals

    return count


def get_all_sources(results: List[Dict]) -> List[str]:
    """
    Get unique list of all sources used.

    Args:
        results: List of result dictionaries

    Returns:
        List of unique source filenames
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create an empty set (sets automatically keep unique values)
    # 2. For each result:
    #    - Get the 'sources' list
    #    - Add each source to the set
    # 3. Convert set to list and return

    # Your code here:
    all_sources = set()
    # Collect all sources

    return list(all_sources)


print("✓ Evaluation functions defined!")

---
## TODO #7: Run Complete Evaluation

**Your Task:** Test your RAG system with all test questions and calculate metrics.

**Python concepts:** Loops, function calls, data aggregation

In [ ]:
def run_evaluation(test_questions: List[Dict]) -> List[Dict]:
    """
    Run RAG system on all test questions.

    Args:
        test_questions: List of test question dictionaries

    Returns:
        List of result dictionaries
    """

    # TODO: Implement this function!
    # HINTS:
    # 1. Create empty list for results
    # 2. For each test question:
    #    - Get the question text
    #    - Call rag_query() with the question
    #    - Add result to results list
    # 3. Return results

    results = []

    # Your code here:
    for test in test_questions:
        # Get question
        # Run RAG query
        # Store result
        pass  # Remove this 'pass' and write your code

    return results


# Run evaluation on all test questions
print("Running evaluation on all test questions...\n")
all_results = run_evaluation(test_questions)

print(f"\n✓ Completed {len(all_results)} tests")

---
## Display Results

Show the evaluation metrics and results.

In [ ]:
# Calculate metrics using your functions
avg_latency = calculate_average_latency(all_results)
successful_retrievals = count_successful_retrievals(all_results)
all_sources_used = get_all_sources(all_results)
hit_rate = successful_retrievals / len(all_results) if all_results else 0

# Display metrics
print_separator("EVALUATION RESULTS")
print(f"\nTotal Questions Tested: {len(all_results)}")
print(f"Successful Retrievals: {successful_retrievals}")
print(f"Hit Rate: {hit_rate:.2%}")
print(f"Average Latency: {avg_latency:.2f} seconds")
print(f"\nSources Used: {', '.join(all_sources_used)}")
print_separator()

In [ ]:
# Display individual results
print("\nIndividual Test Results:\n")

for i, result in enumerate(all_results, 1):
    print(f"[Test {i}]")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer'][:200]}...")
    print(f"Sources: {', '.join(set(result['sources']))}")
    print(f"Time: {result['time']:.2f}s")
    print("-" * 60)
    print()

---
## Save Your Results

Save your test results to a JSON file for your report.

In [ ]:
# Save results to JSON file
results_summary = {
    'metrics': {
        'total_questions': len(all_results),
        'successful_retrievals': successful_retrievals,
        'hit_rate': hit_rate,
        'average_latency': avg_latency
    },
    'results': all_results
}

with open('evaluation_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✓ Results saved to 'evaluation_results.json'")

---
## Congratulations! 🎉

You've successfully built a RAG system!

### What You Accomplished:
✅ Loaded documents from files  
✅ Chunked text with overlap  
✅ Created a RAG query pipeline  
✅ Built a test dataset  
✅ Calculated evaluation metrics  
✅ Generated a results report  

### Next Steps:
- Try different chunk sizes and overlaps
- Add more test questions
- Experiment with different values for `top_k`
- Analyze which questions work best
- Write up your findings in a report

### For Your Report:
1. Describe your document collection
2. Explain your chunking strategy
3. Present your evaluation metrics
4. Show examples of good and bad answers
5. Discuss what you learned

Great job! 🚀